In [13]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import os
import time
from datetime import date, timedelta
from ipywidgets import *
from  bqwidgets import AutoComplete
import ipywidgets
import bqplot
from bqplot import ColorScale, Tooltip
import importlib
import multiple_select

importlib.reload(multiple_select) 

bq = bql.Service()
d = bq.data
f = bq.func

In [14]:
# PAY_RANK_MAP = { '1.5 Lien':'Senior',
#                 '1st lien':'Senior',
#                 '2nd lien':'Sub',
#                 '3rd lien':'Sub',
#                 'Asset Backed':'Senior',
#                 'Jr Subordinated':'Sub',
#                 'Secured':'Senior',
#                 'Sr Subordinated':'Sub',
#                 'Sr Unsecured':'Sub',
#                 'Subordinated':'Sub',
#                 'Unsecured':'Senior'}

curr_widgets_version = -1
DEFAULT_IDX = 'ERL0 Index, HEC0 Index'
MATURITY_LABELS = np.concatenate((np.array([0,0.5]),np.arange(1,10),np.arange(10,35,5)), axis=0) #All maturity levels
BOX_WIDTH = '26.66%'
LBL_WIDTH = '6.66%'
LAYOUT_LBL = Layout(width = '50px')
LAYOUT_BOX = Layout(width = BOX_WIDTH)

data_all_bonds = None
cache_data = None

# check ipywidgets version
a = ipywidgets.__version__.split('.')
curr_widgets_version = int(a[0]) + int(a[1]) / 10


GLOBAL_TEMP = None

#FUNCTIONS    ********************

def label_maturity(x,L):
    length=len(L)
    if x >= L[length-1] or x < 0 or np.isnan(x) == True: 
        label='OUT'
    else:
        for i in range(length):
            if x <= L[i] and x >= L[i-1] and i>=0:
                y=[(x-L[i-1]),(L[i]-x)]
                label=L[i-(1-y.index(min(y)))]
    return label

def bq_res_array_to_pd(res_array):
    a = res_array[0].df().reset_index()
    r_all = pd.DataFrame({'ID':a['ID']})
    for res in res_array:
        r_n = res.df().reset_index()
        r_all = r_all.merge(r_n, on = 'ID')
    return r_all

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    return (d * w).sum() / w.sum()

def WorstRating(ft, sp, md):
    
    FITCH = {'AAA': 23, 'AA+': 22, 'AA': 21, 'AA-': 20, 'A+': 19, 'A': 18, 'A-': 17, 'BBB+': 16, 'BBB': 15, 'BBB-': 14, 'BB+': 13, 'BB': 12, 'BB-': 11, 'B+': 10, 'B': 9, 'B-': 8, 'CCC+': 7, 'CCC': 6, 'CCC-': 5, 'CC+': 4, 'CC': 3, 'CC-': 2, 'DDD': 1}
    SP = {'AAA': 23, 'AA+': 22, 'AA': 21, 'AA-': 20, 'A+': 19, 'A': 18, 'A-': 17, 'BBB+': 16, 'BBB': 15, 'BBB-': 14, 'BB+': 13, 'BB': 12, 'BB-': 11, 'B+': 10, 'B': 9, 'B-': 8, 'CCC+': 7, 'CCC': 6, 'CCC-': 5, 'CC': 4, 'C': 3, 'D': 1}
    inv_SP = {'23': 'AAA', '22': 'AA+', '21': 'AA', '20': 'AA-', '19': 'A+', '18': 'A', '17': 'A-', '16': 'BBB+', '15': 'BBB', '14': 'BBB-', '13': 'BB+', '12': 'BB', '11': 'BB-', '10': 'B+', '9': 'B', '8': 'B-', '7': 'CCC+', '6': 'CCC', '5': 'CCC-', '4': 'CC', '3': 'C', '1': 'D'}
    MOODYS = {'Aaa': 23, 'Aa1': 22, 'Aa2': 21, 'Aa3': 20, 'A1': 19, 'A2': 18, 'A3': 17, 'Baa1': 16, 'Baa2': 15, 'Baa3': 14, 'Ba1': 13, 'Ba2': 12, 'Ba3': 11, 'B1': 10, 'B2': 9, 'B3': 8, 'Caa1': 7, 'Caa2': 6, 'Caa3': 5, 'Ca': 4, 'D': 1}
    
    worst_rtg = []
    length=len(ft)
    for i in range(length):
        
        value_ft = value_ratings(ft[i], FITCH)
        value_sp = value_ratings(sp[i], SP)
        value_md = value_ratings(md[i], MOODYS)
        
        min_rtg = min(value_ft, value_sp, value_md)
        
        if min_rtg == 1000:            
            worst_rtg.append('NA')            
        else:            
            worst_rtg.append(inv_SP[str(min_rtg)])
    
    return worst_rtg
            
def value_ratings(rating, list_ratings):
    
    if rating in list_ratings.keys():
        value = list_ratings[rating]
    else:
        value = 1000
    
    return value

# ********************************************************************************
#
#     Main data cache
#
# ********************************************************************************

def get_data(index):
    global data_all_bonds
    global bq
    global rank_YldorASW
    global request_button1
    global request_button2
    
    # Define fields
    sector = d.BICS_LEVEL_2_INDUSTRY_GROUP_NAME() #BICS_LEVEL_1_SECTOR_NAME
    issue_type = d.PAYMENT_RANK()
    country = d.cntry_of_risk()
    name = d.long_comp_name()
    coupon = d.CPN()
    rating_fitch = d.CREDIT_RATING(CREDIT_RATING_SOURCE  = 'FITCH')
    rating_sp = d.CREDIT_RATING(CREDIT_RATING_SOURCE  = 'SANDP')
    rating_mood = d.CREDIT_RATING(CREDIT_RATING_SOURCE  = 'MOODY')     

    #Constructing DataFrame from a dictionary
    flds = {'sector':sector, 
            'country': country, 
            'issue_type': issue_type, 
            'name': name,
            'coupon': coupon,
            'Rating_Fitch': rating_fitch, 
            'Rating_SP': rating_sp, 
            'Rating_Moodys': rating_mood}

    # Request data
    univ = bq.univ.members(index)
    request = bql.Request(univ,flds)
    response = bq.execute(request)
    data_all_bonds = bq_res_array_to_pd(response)
    
    # calculate worst rating
    data_all_bonds['Worst_RTG'] = WorstRating(data_all_bonds['Rating_Fitch'].tolist(),
                                              data_all_bonds['Rating_SP'].tolist(),
                                              data_all_bonds['Rating_Moodys'].tolist())
    # calculate issue type
#     data_all_bonds['issue_type'] = data_all_bonds['issue_type'].map(PAY_RANK_MAP)
    
    # update gui
    request_button1.description = 'Actualizado'
    request_button1.button_style = 'primary'
    rank_YldorASW.disabled = False
    request_button2.disabled = False  
    
    # cache new download
    data_all_bonds.to_csv('TEMP\CACHE_Data.csv')
    
def get_cache_data(tgt = None):
    """ call back for first button is pressed, this triggers the 
    process of checking if required data cache exists"""

    #Clear the outputs
    reset_output()    
    
    #Reset Tenors
    tenor_select.value = [0.5, 35]
    
    # show indicator in gui that data is downloading
    toggle_bussy()
    
    # read selected index list
    idx = read_index()
    
    #get data
    get_data(idx)
    
    #load cache
    read_excel_cache()
    
    #update sectors
    update_sector_list_control()
    
    #load issuers' name to find in the AutoComplete
    ac.data = cache_data['name'].unique().tolist()
    
    toggle_bussy()

# ********************************************************************************
#
#     selected sector data
#
# ********************************************************************************

def get_sector_info(tgt = None):
    """ call back for when a sector and yield/spread is selected, and the button is pressed 
    
    """
    global ax_y
    
    #Clear the outputs
    reset_output()
    
    # show indicator in gui that data is downloading
    toggle_bussy()
    
    # get inputs
    idx = read_index()
    info_Yld_o_ASW = rank_YldorASW.value
    
    info_sector = accordion_sector_ms.read_values()
    
    #get data -> saved to Global_TEMP
    get_sector_data(idx, info_Yld_o_ASW, info_sector)
    
    #update filters
    update_country_list_control()
    update_rating_list_control()
    update_issue_type_list_control()
    update_issuer_list_control(info_sector)
    
    #Define y axis label
    ax_y.label = rank_YldorASW.value
    
    #Reset Tenors
    tenor_select.value = [0.5, 35]
    
    #Reset table filters
    check_name.value = True
    check_mat.value = True
    preference1.value = 'ABC'
    preference2.value = 'Descendente'
    
    #Calculate sector average and plot the curve -> TEMP
    get_sector_curve()
    
    # hide indicator in gui that data is downloading
    toggle_bussy() 

def get_sector_data(idx, yld_or_sprd, sector):
    # the main index cache for positions, ratings
    global cache_data
    # the variable to which we save the current sector info
    global GLOBAL_TEMP
    
    # filter index members to selected sector
    aux_idx = bq.univ.members(idx)
    criteria =  cache_data.loc[cache_data['sector'].isin(sector)]['ID'].tolist()
    filtered_univ = bq.univ.filter(aux_idx, f.in_(d.id(), criteria))
    
    # define fields to download
    ticker = d.id()
    weight = ticker['Weights'] / 100
    # if the type of the issue is 'Subordinated', 'Jr Subordinated' or 'Sr Subordinated', the maturity must be taken with the field NXT_CALL_DT() 
    issue_type = d.PAYMENT_RANK()
    matur = f.if_(f.or_(issue_type == 'Subordinated', f.or_(issue_type == 'Jr Subordinated', issue_type == 'Sr Subordinated')), d.NXT_CALL_DT(), d.maturity())
    
    if yld_or_sprd == 'Yield':
        sel_info = d.yield_(fill = 'prev')
#         sel_info =  f.fillna(f.if_(sel_info == 0, np.nan, sel_info), limit = 10, on = 'date')
    else:
        sel_info = d.spread(spread_type = 'ASW')
    
    name2 = d.name()
    flds = {'weight': weight, 'maturity': matur, 'info': sel_info, 'name2': name2}
    
    # make request, request data
    request =  bql.Request(filtered_univ, flds)
    response = bq.execute(request)
    
    # create cache with requested data
    TEMP = bq_res_array_to_pd(response)
    GLOBAL_TEMP = pd.merge(TEMP, cache_data, how='left', on=['ID', 'ID'])
    
    # update gui
    accordion_countries_ms.update_values(GLOBAL_TEMP['country'], True) 
    accordion_ratings_ms.update_values(GLOBAL_TEMP['Worst_RTG'], True) 
    accordion_issue_type_ms.update_values(GLOBAL_TEMP['issue_type'], True) 
    

#****************************************************************
#
#      Chart sector curve
#
#****************************************************************

def get_sector_curve(tgt = None):
    # cache of the sector data
    global GLOBAL_TEMP
    global line_sector
    
    TEMP = GLOBAL_TEMP.copy(deep=True)
    
    #Take only the data filtered in the accordions
    country_list = accordion_countries_ms.read_values()
    issue_t_list = accordion_issue_type_ms.read_values()
    ratings_list = accordion_ratings_ms.read_values()
    
    # filter Data
    TEMP = TEMP[TEMP.country.isin(country_list)]
    TEMP = TEMP[TEMP.issue_type.isin(issue_t_list)]
    TEMP = TEMP[TEMP.Worst_RTG.isin(ratings_list)]
    
    #Change maturity to number of years
    get_mat_labels(TEMP)
    TEMP['Maturity_Labels'] = TEMP['Maturity_Labels'].apply(pd.to_numeric, errors='coerce')
    
    min_tenor = tenor_select.value[0]
    max_tenor = tenor_select.value[1]
    tgt_maturity = (TEMP['Maturity_Labels'] >= min_tenor) & (TEMP['Maturity_Labels'] <= max_tenor)
    TEMP = TEMP[tgt_maturity]
    overview = TEMP.groupby(['Maturity_Labels']).apply(wavg, 'info', 'weight')
    
    overview = pd.DataFrame([overview]).T.reset_index()
    overview.columns = ['Maturity_Labels', 'vals']
    
    #Reset y axis values
    sc_y = bqplot.LinearScale()
    
#     Plot the result
    line_sector.x = []
    line_sector.y = []
    
    x_data = overview['Maturity_Labels'].tolist()
    line_sector.x = x_data
    line_sector.y = overview['vals'].tolist()
    line_sector.labels = ['Media Sectorial']
    
    tenor_select.value = [min(x_data), max(x_data)]

def get_mat_labels(TEMP):
    #Change maturity to number of years
    TEMP.maturity = ((pd.to_datetime(TEMP.maturity) - date.today()) / 365) / timedelta(days = 1)
    
    mat = TEMP['maturity'].tolist()
    labels = []
    length = len(mat)
    for i in range(length):
        labels.append(label_maturity(mat[i], MATURITY_LABELS))
    
    TEMP['Maturity_Labels'] = labels
    
    # 2018-10-17 Ryo si no hay 'OUT' pandas da error type conversion.
    if 'OUT' in labels:
        TEMP = TEMP[TEMP.Maturity_Labels != 'OUT']
    
def read_index():
    input_idx = index_input.value
    cur_year = str(date.today().year)
    cur_month = str(date.today().month)
    
    #save index name and properties in the txt CACHE
    text_file = open('TEMP\CACHE_info.txt', 'w+')
    text_file.write('INDICE(S): -'+ input_idx + '-\nAÑO: -'+ cur_year + '-\nMES: -' + cur_month)
    text_file.close()
    
    return list(input_idx.split(', '))

# ********************************************************************************
#
#     Read Caché
#
# ********************************************************************************

def read_excel_cache():
    global cache_data
    
    cache_data = pd.read_csv('TEMP\CACHE_Data.csv')
    cache_data = cache_data.drop(['Unnamed: 0'], axis=1)
    
def print_ac_properties(tgt = None):
    try:
        list_sectors = cache_data[cache_data['name'] == ac.value]['sector'].unique().tolist()
        if len(list_sectors) > 1:
            aux_sec = ', '.join(list_sectors)
        else:
            aux_sec = list_sectors[0]

        ac_output.value = 'SECTOR: ' + aux_sec
    except:
        ac_output.value = ''    
    
# ******************************************************************
#
#     GUI functions
#
#******************************************************************

def reset_output(tgt = None):
    table_filters.layout.visibility = 'hidden'
    select_issuers.options = []
    tbl.layout.visibility = 'hidden' 

# ********************************************************************************
#     get sector name list
# ********************************************************************************

def update_sector_list_control():
    global cache_data
    global accordion_sector_ms
    
    aux_sectors = list(cache_data['sector'].unique())
    aux_sectors.sort()
    
    if None in aux_sectors:
        aux_sectors.remove(None)
        
    accordion_sector_ms.update_values(aux_sectors, False)
    
# ********************************************************************************
#     get rating name list
# ********************************************************************************

def update_rating_list_control():
    global GLOBAL_TEMP
    global accordion_ratings
    
    aux_rating = list(GLOBAL_TEMP['Worst_RTG'].unique())
    aux_rating.sort()
    if None in aux_rating:
        aux_rating.remove(None)
    accordion_ratings_ms.update_values(aux_rating, True)
    
    
# ********************************************************************************
#     get country name list
# ********************************************************************************

def update_country_list_control():
    global GLOBAL_TEMP
    global accordion_countries
    
    aux_country = list(GLOBAL_TEMP['country'].unique())
    aux_country.sort()
    if None in aux_country:
        aux_country.remove(None)
    accordion_countries_ms.update_values(aux_country, True)

# ********************************************************************************
#     get issue type list
# ********************************************************************************

def update_issue_type_list_control():
    global GLOBAL_TEMP
    global accordion_issue_type
    
    issue_type  = list(GLOBAL_TEMP['issue_type'].unique())
    issue_type.sort()
    if None in issue_type:
        issue_type.remove(None)
    accordion_issue_type_ms.update_values(issue_type, True)
    
    
# ********************************************************************************
#
#     get issuer name list
#
# ********************************************************************************

def update_issuer_list_control(sector, countries = None, ratings = None, rank = None):
    global GLOBAL_TEMP
    global select_issuers
    
    # Filter issuer list
    sector_issuers = GLOBAL_TEMP[GLOBAL_TEMP['sector'].isin(sector)]
    if not countries == None:
        sector_issuers = sector_issuers[sector_issuers['country'].isin(countries)]
    if not ratings == None:
        sector_issuers = sector_issuers[sector_issuers['Worst_RTG'].isin(ratings)]
    if not rank == None:
        sector_issuers = sector_issuers[sector_issuers['issue_type'].isin(rank)]
    issuers  = list(sector_issuers['name'].unique())
    issuers.sort()
    if None in issuers:
        issuers.remove(None)
    select_issuers.options = issuers   
    
def toggle_bussy():
        a = wait_msg
        b = fig_line
        c = tenor_select
        if a.layout.visibility == 'hidden':
            a.layout.visibility = 'visible'
            a.layout.height = '100%'
            b.layout.visibility = 'hidden'
            c.layout.visibility = 'hidden'
        else:
            a.layout.visibility = 'hidden'
            a.layout.height = '1px'
            b.layout.visibility = 'visible'
            c.layout.visibility = 'visible'
            
def generate_HTML_tbl(df):    
    col_widths = [1/9]*9

    html_txt = "<table class=\"table table-bordered\" style=\"display:block;\">"

    #     Headers
    headers = df.columns.values
    html_txt = html_txt + "<thead><tr>"

    for h, c in zip(headers, col_widths):
        th = '<th style =\"width:' + str(c) +'% \">'
        th = th + '<div style=\"overflow:hidden;min-height:20px; max-height:20px;\"><b>'
        th = th + h 
        th = th + '</b></div>'
        th = th + '</th>'
        html_txt = html_txt + th
    html_txt = html_txt + "</tr>"       
    html_txt = html_txt + "</thead>"

    # Body
    df_vals = df.values
    html_txt = html_txt + "<tbody>"
    for r in df_vals:
        html_txt = html_txt + "<tr>"
        for v, w in zip(r, col_widths):
            html_txt = html_txt + '<td  style =\"width:' + str(w) +'% \" >'
            html_txt = html_txt + '<div style=\"overflow:hidden;min-height:20px; max-height:20px; color:white\">'
            html_txt = html_txt + str(v)
            html_txt = html_txt + '</div>'
            html_txt = html_txt + '</td>'
        html_txt = html_txt + '</tr>'

    html_txt = html_txt + "</tbody></table>"
    
    return html_txt    
            
# rank dropdown
def accordion_call_back(tgt = None):
    
    country_list = accordion_countries_ms.read_values()
    if len(country_list) == 0:
        country_list = None
        
    issue_t_list = accordion_issue_type_ms .read_values()
    if len(issue_t_list) == 0:
        issue_t_list = None
        
    ratings_list = accordion_ratings_ms.read_values()
    if len(ratings_list) == 0:
        ratings_list = None
    
    sector = accordion_sector_ms.read_values()
    
    update_issuer_list_control(sector, country_list, ratings_list, issue_t_list )
    
    #Reset Tenors
    tenor_select.value = [0.5, 35]
    
    get_sector_curve()
    
    #Reset table filters and hide them
    check_name.value = True
    check_mat.value = True
    preference1.value = 'ABC'
    preference2.value = 'Descendente'
    
    table_filters.layout.visibility = 'hidden'
        
def build_accordion(name, width, margin, cb_func):
    accordion = Accordion()
    accordion.set_title(0, name)
    accordion.layout.width = BOX_WIDTH

    if curr_widgets_version >= 7.3:
        accordion.selected_index = None
    else:
        accordion.selected_index = -1
    
    ms = multiple_select.MultipleSelect(width, margin, cb_func)
    accordion.children = [ms.panel]
    return accordion, ms

def issuer_select_event_handler(tgt = None):
    global iss_curves, iss_points, color_data, fil
    
    iss_curves = dict()
    iss_points = dict()
    a = list(select_issuers.value)
    color_data = list(range(len(a)))
    
    count=0
    for a1 in a:
        chart_issuer_curve(a1, count)
        count += 1
        
    #Plot issuers' curves
    fig_line.marks = [line_sector] + list(iss_curves.values()) + list(iss_points.values())
    
    #Generate the HTML table
    fil = GLOBAL_TEMP.copy(deep=True)
    fil = fil.drop(['Rating_Fitch', 'Rating_SP', 'Rating_Moodys', 'DATE'], axis = 1)
    fil = fil[fil.name.isin(a)]
    
    #Take only the data filtered in the accordions
    country_list = accordion_countries_ms.read_values()
    issue_t_list = accordion_issue_type_ms .read_values()
    ratings_list = accordion_ratings_ms.read_values()
    
    # filter Data
    try:
        fil = fil[fil.country.isin(country_list)]
    except:
        None
    try:
        fil = fil[fil.issue_type.isin(issue_t_list)]
    except:
        None
    try:
        fil = fil[fil.Worst_RTG.isin(ratings_list)]
    except:
        None
    
#     fil = fil.sort_values(by = ['name', 'maturity'])
    info = rank_YldorASW.value
    fil = fil.rename(index = str, columns = {'ID': 'Ticker', 'name': 'Emisor', 'coupon': 'Cupón', 'weight': 'Peso (Base 100)','maturity': 'Vencimiento','info': info, 'country': 'País', 'Worst_RTG':'Rating', 'issue_type': 'Tipo'})
    fil = fil[['Ticker', 'Emisor', 'Cupón','Vencimiento', 'Peso (Base 100)', info, 'País', 'Rating','Tipo']]
    fil['Peso (Base 100)'] = (fil['Peso (Base 100)']*100).round(4)
    fil[info] = fil[info].round(2)
    fil['Vencimiento'] = fil['Vencimiento'].dt.strftime('%d/%m/%Y')
    
    sort_selected_table()
    tbl.value = generate_HTML_tbl(fil)
    tbl.layout.visibility = 'visible'
    
    table_filters.layout.visibility = 'visible'
    
def sort_selected_table(tgt = None):
    global fil
    
    sort_flds = []
    ascending_flds = []
    if check_name.value == True:
        sort_flds.append('Emisor')
        
        if preference1.value == 'ABC':
            ascending_flds.append(True)
        else:
            ascending_flds.append(False)        
    
    if check_mat.value == True:
        sort_flds.append('Vencimiento')
        
        if preference2.value == 'Ascendente':
            ascending_flds.append(False)
        else:
            ascending_flds.append(True)
            
    fil = fil.sort_values(by = sort_flds, ascending = ascending_flds)
    
def print_sort_table(tgt = None):
    sort_selected_table()
    tbl.value = generate_HTML_tbl(fil)
    tbl.layout.visibility = 'visible'

def chart_issuer_curve(issuer_name, count):
    aa = GLOBAL_TEMP.copy(deep=True)
    
    #Take only the data filtered in the accordions
    country_list = accordion_countries_ms.read_values()
    issue_t_list = accordion_issue_type_ms .read_values()
    ratings_list = accordion_ratings_ms.read_values()    
    
    # filter Data
    try:
        aa = aa[aa.country.isin(country_list)]
    except:
        None
    try:
        aa = aa[aa.issue_type.isin(issue_t_list)]
    except:
        None
    try:
        aa = aa[aa.Worst_RTG.isin(ratings_list)]
    except:
        None    
    aa = aa[aa['name'] == issuer_name]
    aa['maturity_years'] = ((pd.to_datetime(aa['maturity']) - date.today()) / 365) / timedelta(days = 1)

    aa = aa.sort_values('maturity_years')

    min_tenor = tenor_select.value[0]
    max_tenor = tenor_select.value[1]
        
    aa = aa[(aa['maturity_years'] >= min_tenor) & (aa['maturity_years'] <= max_tenor)]

    #Plot the result
    iss_curves[issuer_name] = bqplot.Lines(x = aa['maturity_years'].tolist(), y = aa['info'].tolist(),
                                scales={'x': sc_x, 'y': sc_y, 'color': sc_col}, display_legend=True, color = [color_data[count]], marker = 'triangle-down', labels = [issuer_name])
    
    def_tt = Tooltip(fields=['name'], formats=[''], show_labels = False)
    
    iss_points[issuer_name] = bqplot.Scatter(x = aa['maturity_years'].tolist(), y = aa['info'].tolist(),
                                scales={'x': sc_x, 'y': sc_y}, default_opacities = [0],
                                            names = aa['name2'].tolist(), display_names = False, tooltip = def_tt)
        
def tenor_slider_callback_handler(tgt = None):
    global iss_curves, iss_points, color_data
    
    get_sector_curve()
    iss_curves = dict()
    iss_points = dict()
    a = list(select_issuers.value)
    color_data = list(range(len(a)))
    
    count=0
    for a1 in a:
        chart_issuer_curve(a1, count)
        count += 1
    
    fig_line.marks = [line_sector] + list(iss_curves.values()) + list(iss_points.values())
    sc_x.min = tenor_select.value[0]
    sc_x.max = tenor_select.value[1]
    
def index_update(tgt = None):
    try:
        text_file = open('TEMP\CACHE_info.txt', 'r')
        saved_index = text_file.read()
        saved_index = list(saved_index.split('-'))
        
        if saved_index[1] != index_input.value or is_updated  == False:          
            request_button1.description = 'ACTUALIZAR'   
            request_button1.button_style = 'danger'
        else:
            request_button1.description = 'Actualizado'
            request_button1.button_style  = 'primary'
            
    except:
        None

# ************************************************************
#    Please wait message
# ************************************************************

wait_msg = HTML("""<h2 style="color:white;">Dercargando datos .....</h>
    <i class="fa fa-spinner fa-spin fa-2x fa-fw" style="color:white;"></i>""")
wait_msg.layout.height = '1px'
wait_msg.layout.visibility = 'hidden'
wait_msg.layout.margin = '20px 10px 10px 10px'

In [15]:
#GUI    ********************
  
aux_label = Label(value='')
aux_label.layout.width = LBL_WIDTH
    
# ************************************************************
#    Row 1 - index inputs
# ************************************************************

# ************************************************************
#    Row 2 - Autocomplete -> Show issuer's properties
# ************************************************************

label_ac = Label(value='Busca Emisor')
ac = AutoComplete(data = [])
ac.observe(print_ac_properties, names = 'value')

ac_output = Text(value = '', disabled=True)
ac_output.layout.width = '100px'

label_ac.layout.width = LBL_WIDTH
ac.layout.width = BOX_WIDTH
ac_output.layout.width = '59.99%'

inputs_top_row_2_0 = HBox([label_ac, ac,aux_label, ac_output])
          
# ************************************************************
#    Row 3 - filters
# ************************************************************

# rank dropdown

rank_YldorASW = Dropdown(options = ['Yield', 'ASW'])
accordion_sector, accordion_sector_ms = build_accordion('Sectores', '100%', '1px', accordion_call_back)
request_button2 = Button(description = 'VER', button_style = 'warning')
request_button2.on_click(get_sector_info)

rank_YldorASW.layout.width = BOX_WIDTH
request_button2.layout.width = BOX_WIDTH
rank_YldorASW.layout.height = '43px'
request_button2.layout.height = '43px'
rank_YldorASW.disabled = True
request_button2.disabled = True

# pack
inputs_top_row_2 = HBox([aux_label, rank_YldorASW, aux_label, accordion_sector, aux_label, request_button2])

# Index picker
label_universe = Label(value='Indice(s)')
index_input = Text(value = '')

# button download data
try:
    text_file = open('TEMP\CACHE_info.txt', 'r')
    global_info = text_file.read()
    global_info = list(global_info.split('-'))
    
    index_input.value = global_info[1]
    cur_date = [str(date.today().year), str(date.today().month)]
    file_date = [global_info[3], global_info[5]]
    is_updated = cur_date == file_date
    
except:
    index_input.value = DEFAULT_IDX
    is_updated = False

index_input.observe(index_update, names = 'value')

request_button1 = Button(description = '')
request_button1.on_click(get_cache_data)

# pack
inputs_top_row_1 = HBox([label_universe, index_input, aux_label, request_button1])


label_universe.layout.width = LBL_WIDTH
index_input.layout.width = BOX_WIDTH
request_button1.layout.width = '59.99%'

# ************************************************************
#    Row 4 - filters
# ************************************************************


# ************************************************************
#   accordions
# ************************************************************
accordion_countries, accordion_countries_ms = build_accordion('País', '20%', '1px', accordion_call_back)
accordion_ratings, accordion_ratings_ms = build_accordion('Rating', '25%', '1px', accordion_call_back)
accordion_issue_type, accordion_issue_type_ms = build_accordion('Tipo', '100%', '1px', accordion_call_back)
    
# pack
inputs_top_row_3 = HBox([aux_label, accordion_countries, aux_label, accordion_ratings, aux_label, accordion_issue_type])

# ************************************************************
#   OUTPUT Row 5 - Plot and filtered issuer list
# ************************************************************

#Issuer list selection
label_select_issuers = Label(value='Lista Emisores')
select_issuers = SelectMultiple(options = [], rows=10)
select_issuers.layout.width = '100%'
select_issuers.layout.height = '90%'
select_issuers.observe(issuer_select_event_handler, names = 'value')

output_box1_1 = VBox([label_select_issuers, select_issuers])


# ************************************************************
#   Chart
# ************************************************************

#Defining the axis types.
sc_x = bqplot.LinearScale()
sc_y = bqplot.LinearScale()
sc_col = ColorScale(colors=['Red', 'Green'])

#Defining the chart type.
line_sector = bqplot.Lines(x = [], y = [], scales={'x': sc_x, 'y': sc_y}, display_legend=True, colors=['orange'], marker = 'circle')

#Defining axis parameters. 
#num_ticks is useful to reduce clutter in the axis, 
ax_x = bqplot.Axis(scale = sc_x, label = 'Vencimientos')
ax_y = bqplot.Axis(scale=sc_y, orientation = 'vertical', tick_format = '0.2f', label = rank_YldorASW.value)

fig_line = bqplot.Figure(marks = [line_sector], axes = [ax_x, ax_y], legend_location = 'bottom')
fig_line.fig_margin = dict(top = 5, bottom = 20, left = 50, right = 5)
fig_line.layout.width = '95%'


#Term selection slider
min_tenor = 0.5
max_tenor = 35
tenor_select = IntRangeSlider(value = [min_tenor, max_tenor],  min = min_tenor, max = max_tenor)
tenor_select.layout.width = '100%'
tenor_select.observe(tenor_slider_callback_handler, names = 'value')

output_box1_2 = VBox([wait_msg, fig_line, tenor_select])

#pack
output_box1 = HBox([output_box1_1, output_box1_2])
output_box1_1.layout.width = '33.33%'
output_box1_1.layout.overflow_x = 'hidden'

output_box1_2.layout.width = '66.66%'
# output_box1_2.layout.width = '59.99%'
output_box1_2.layout.overflow_x = 'hidden'

# ************************************************************
#   OUTPUT Row 6 - Issuer's info HTML table
# ************************************************************

label_check = Label(value='Campos')
check_name = Checkbox(value=True, description='Emisor')
check_mat = Checkbox(value=True, description='Vencimiento')
label_pref = Label(value='Orden')
preference1 = Dropdown(options=['ABC', 'ZYX'], value='ABC')
preference2 = Dropdown(options=['Ascendente', 'Descendente'], value='Descendente')
sort_button = Button(description = 'ORDENAR', button_style = 'success')
sort_button.layout.height = '60px'
sort_button.on_click(print_sort_table)

table_filters = HBox([VBox([label_check, label_pref]), VBox([check_name, preference1]), VBox([check_mat, preference2]), sort_button])
table_filters.layout.visibility = 'hidden'

tbl = HTML('')
tbl.layout.visibility = 'hidden'
tbl.layout.width = '98%'
tbl.layout.height = '200px'

# ************************************************************
#    Main canvas
# ************************************************************
aux_label2 = Label(value='')
aux_label2.layout.height = '3px'

main_canvas = VBox([inputs_top_row_1, aux_label2, inputs_top_row_2_0, aux_label2, inputs_top_row_2, aux_label2, inputs_top_row_3, aux_label2, output_box1, aux_label2, table_filters, aux_label2, tbl])
main_canvas.layout.width = '1400px'

if is_updated == True:
    request_button1.description = 'Actualizado'
    request_button1.button_style  = 'primary'
    rank_YldorASW.disabled = False
    request_button2.disabled = False
    read_excel_cache()
    update_sector_list_control()
    ac.data = cache_data['name'].unique().tolist()    
else:
    request_button1.description = 'ACTUALIZAR'   
    request_button1.button_style = 'danger'
    rank_YldorASW.disabled = True
    request_button2.disabled = True
    
debug_data = None    
main_canvas